Prepare the data
- Remove the columns with spurious data:
- Remove the rows with only zeros
- Rename the genes to be sure there is a map with Entrez Gene Identifers
- Rename ages
- Extract the metadata
- Export the counts and metadata

*Generated from a Jupyter Notebook - [Sources](https://github.com/bebatut/neuromac_GF_mices/blob/master/src/prepare_data.ipynb)*

# Load

In [1]:
source("load_libraries.R")

Warning message:
“package ‘gplots’ was built under R version 4.1.3”
Warning message:
“package ‘WGCNA’ was built under R version 4.1.3”


Allowing multi-threading with up to 8 threads.


Warning message:
“package ‘reshape’ was built under R version 4.1.3”
preparing gene to GO mapping data...

preparing IC data...

preparing gene to GO mapping data...

preparing IC data...

preparing gene to GO mapping data...

preparing IC data...



In [2]:
source("functions.R")

In [3]:
dp = "../results/dge/"
dir.create(dp, showWarnings = FALSE)

Load the gene length

In [4]:
gene_length = read.table("../data/gene_length.tabular", h = F, row.names = 1)
save(gene_length, file=paste(dp, "gene_length.RData", sep=''))

Load the count table

In [5]:
counts = read.table("../data/counts.tabular", sep="\t", header = 1, row.names=1)
dim(counts)

[1] 24421    64

# Extract metadata

Extract the description of the samples

In [6]:
file_desc = read.csv("../data/file_description.csv", row.names = 1) %>%
            rownames_to_column('sample') %>%
            slice(1:(n()-6)) %>%
            mutate(Lane = gsub(" & ", "_", Lane)) %>%
            mutate(Lane = gsub(" ", "1", Lane)) %>%
            mutate(Project.id = gsub("Project_", "", Project.id)) %>%
            mutate(Project.id = gsub("148", "S148", Project.id))
dim(file_desc)
head(file_desc)

[1] 64 10

,sample,Sample.name.prefix,Group,Age,Gender,Project.id,Lane,Replicate,Name.in.project,X..Reads
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
1,GF_52w_M_1,GF_52w_M,GF,52w,M,S148,2,1,Sample_Mg_GF1_old,"35,713,942.00"
2,GF_8w_M_1,GF_8w_M,GF,8w,M,S148,2,1,Sample_Mg_GF1_young,"25,919,398.00"
3,GF_52w_M_2,GF_52w_M,GF,52w,M,S148,2,2,Sample_Mg_GF2_old,"29,752,263.00"
4,GF_8w_M_2,GF_8w_M,GF,8w,M,S148,2,2,Sample_Mg_GF2_young,"24,133,081.00"
5,GF_52w_M_3,GF_52w_M,GF,52w,M,S148,3,3,Sample_Mg_GF3_old,"26,395,568.00"
6,GF_8w_M_3,GF_8w_M,GF,8w,M,S148,1,3,Sample_Mg_GF3_young,"33,992,780.00"


Extract the metadata:
1. Extract from the counts
2. Combine with the sample descriptions
3. Rename the ages
  - 8w to Young
  - 52w to Middle-aged
  - 104w to Old
4. Rename the gender
  - F to Female
  - M to Male
5. Rename the factors
  - gender to Sex
  - type to Microbiota
  - age to Age

In [7]:
metadata = as_tibble(t(sapply(sapply(colnames(counts), strsplit, split = "_"), unlist))) %>%
   magrittr::set_colnames(c("Microbiota", "Age", "Sex", "Replicate", "extra")) %>%
   mutate(sample = paste(Microbiota, Age, Sex, Replicate, extra, sep ="_")) %>%
   select(-extra) %>%
   mutate(short_name = paste(Microbiota, Age, Sex, Replicate, sep ="_")) %>%
   select(-Replicate) %>%
   arrange(short_name) 
projects = file_desc %>%
   filter(sample %in% metadata$short_name) %>%
   arrange(sample) %>%
   pull("Project.id")
metadata = metadata %>%
    mutate(project = projects) %>%
    select(-short_name) %>%
    mutate(Age = gsub("104w", "Old", Age )) %>%
    mutate(Age = gsub("52w", "Middle-aged", Age )) %>%
    mutate(Age = gsub("8w", "Young", Age )) %>%
    mutate(Sex = gsub("F", "Female", Sex )) %>%
    mutate(Sex = gsub("M", "Male", Sex))
metadata %>%
    group_by(Microbiota, Age, Sex)  %>% 
    summarise(Number = n())
metadata

Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
Using compatibility `.name_repair`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated.”
`summarise()` has grouped output by 'Microbiota', 'Age'. You can override using the `.groups` argument.



Microbiota,Age,Sex,Number
<chr>,<chr>,<chr>,<int>
GF,Middle-aged,Female,6
GF,Middle-aged,Male,4
GF,Old,Female,3
GF,Old,Male,5
GF,Young,Female,5
GF,Young,Male,4
SPF,Middle-aged,Female,6
SPF,Middle-aged,Male,5
SPF,Old,Female,3


Microbiota,Age,Sex,sample,project
<chr>,<chr>,<chr>,<chr>,<chr>
GF,Old,Female,GF_104w_F_1_2,S264
GF,Old,Female,GF_104w_F_2_2,S264
GF,Old,Female,GF_104w_F_3_2,S264
GF,Old,Male,GF_104w_M_1_2,S264
GF,Old,Male,GF_104w_M_2_2,S264
GF,Old,Male,GF_104w_M_3_2,S288
GF,Old,Male,GF_104w_M_4_2,S288
GF,Old,Male,GF_104w_M_5_2,S288
GF,Middle-aged,Female,GF_52w_F_1_2,S178


# Keep interesting samples

Samples to keep :
- SPF / GF
- young / old
- male

In addition
- Remove the columns with spurious data:
    - SPF_8w_F_2_2: low mapping rate (70.9%) and assignment rate (31.3%)
    - SPF_8w_F_5_2: low assignment rate (53.5%)
    - SPF_8w_M_1_2: low number of assigned reads (14.4)

In [8]:
metadata = metadata %>%
    filter(Sex == 'Male') %>%
    filter(Age != 'Middle-aged') %>%
    filter(!sample %in% c("SPF_8w_F_2_2", "SPF_8w_F_5_2", "SPF_8w_M_1_2")) %>%
    select(-Sex)
metadata

Microbiota,Age,sample,project
<chr>,<chr>,<chr>,<chr>
GF,Old,GF_104w_M_1_2,S264
GF,Old,GF_104w_M_2_2,S264
GF,Old,GF_104w_M_3_2,S288
GF,Old,GF_104w_M_4_2,S288
GF,Old,GF_104w_M_5_2,S288
GF,Young,GF_8w_M_1_2,S148
GF,Young,GF_8w_M_2_2,S148
GF,Young,GF_8w_M_3_2,S148
GF,Young,GF_8w_M_4_2,S148


In [9]:
save(metadata, file=paste(dp, "metadata.RData", sep=''))

In [10]:
counts = counts[,names(counts) %in% metadata$sample]

In [11]:
metadata %>% 
    select(-sample) %>% 
    group_by(Age, Microbiota) %>%
    arrange(project) %>%
    filter(row_number()==1)

Microbiota,Age,project
<chr>,<chr>,<chr>
GF,Young,S148
SPF,Young,S148
SPF,Old,S174
GF,Old,S264


# Remove spurious data

Remove the rows with only zeros (printed: before / after)

In [12]:
nrow(counts)
counts = counts[ rowSums(counts) > 1, ]
nrow(counts)

[1] 24421

[1] 18113

# Rename genes names

Rename the genes to be sure there is a map with Entrez Gene Identifers:

Mapping extracted using:

1. Extract mapping between 
    - Entrez Gene Identifiers and Gene Names (and description) 
    - Gene Symbols and Entrez Gene Identifiers
    - RefSeq and Entrez Gene Identifiers
    - Gene Symbols and Gene Names (and description)
    - Entrez Gene Identifiers and KEGG pathways
2. Get RefSeq ids for genes in count table
3. Get genes without RefSeq ids (printed: number of them that will need to be changed)
4. Search on Entrez the correct name of the genes without RefSeq ids
5. Clean the new names to keep only the ones with one new names, that are not already in the count table,
6. Apply them to the counts
7. Check the new number of genes without RefSeq ids

In [13]:
table = read.table("../data/name_replacement", sep=",", header = 1, row.names = 1)

In [14]:
new_names = rownames(counts)
names(new_names) = rownames(counts)
names_to_replace = new_names %in% rownames(table)
new_names[names_to_replace] = table[new_names[names_to_replace],'New']
rownames(counts) = new_names

# Order the data

Reorder the columns and save the counts object

In [15]:
head(counts)
counts = counts %>%
         rownames_to_column('gene_names') %>%
         select(sort(current_vars())) %>%
         column_to_rownames('gene_names')
head(counts)
save(counts, file=paste(dp, "prepared_counts.RData", sep=''))

,GF_104w_M_1_2,GF_104w_M_2_2,GF_8w_M_1_2,GF_8w_M_2_2,GF_8w_M_3_2,GF_8w_M_4_2,SPF_104w_M_1_2,SPF_104w_M_2_2,SPF_104w_M_3_2,SPF_104w_M_4_2,⋯,SPF_104w_M_8_2,SPF_104w_M_9_2,SPF_104w_M_10_2,SPF_104w_M_11_2,SPF_104w_M_12_2,SPF_104w_M_13_2,SPF_104w_M_14_2,GF_104w_M_3_2,GF_104w_M_5_2,GF_104w_M_4_2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,2,2,7,1,6,7,9,2,2,2,⋯,2,1,9,10,0,0,6,1,1,1
Erg28,1002,1577,499,619,614,758,613,531,713,381,⋯,851,875,1193,941,842,846,786,394,570,496
0610009B22Rik,1385,2475,784,822,974,1019,631,1008,866,611,⋯,1278,1205,2042,1776,1390,1348,1370,551,772,522
0610009L18Rik,23,61,51,13,51,30,25,19,33,25,⋯,50,24,54,64,40,37,52,20,22,26
Dele1,256,432,298,237,520,356,189,271,294,228,⋯,225,192,285,304,108,247,203,269,351,263
Sanbr,315,483,284,546,429,507,166,227,406,223,⋯,340,524,1122,718,576,468,585,412,540,207


Warning message:
“`current_vars()` was deprecated in dplyr 0.8.4.
Please use `tidyselect::peek_vars()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated.”


,GF_104w_M_1_2,GF_104w_M_2_2,GF_104w_M_3_2,GF_104w_M_4_2,GF_104w_M_5_2,GF_8w_M_1_2,GF_8w_M_2_2,GF_8w_M_3_2,GF_8w_M_4_2,SPF_104w_M_1_2,⋯,SPF_104w_M_3_2,SPF_104w_M_4_2,SPF_104w_M_5_2,SPF_104w_M_6_2,SPF_104w_M_7_2,SPF_104w_M_8_2,SPF_104w_M_9_2,SPF_8w_M_2_2,SPF_8w_M_3_2,SPF_8w_M_4_2
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0610005C13Rik,2,2,1,1,1,7,1,6,7,9,⋯,2,2,23,0,1,2,1,9,8,4
Erg28,1002,1577,394,496,570,499,619,614,758,613,⋯,713,381,963,863,1005,851,875,615,648,745
0610009B22Rik,1385,2475,551,522,772,784,822,974,1019,631,⋯,866,611,1639,1377,1174,1278,1205,985,941,1063
0610009L18Rik,23,61,20,26,22,51,13,51,30,25,⋯,33,25,49,44,12,50,24,27,55,76
Dele1,256,432,269,263,351,298,237,520,356,189,⋯,294,228,378,180,266,225,192,493,348,429
Sanbr,315,483,412,207,540,284,546,429,507,166,⋯,406,223,172,437,677,340,524,313,303,355


Load the gene length, keep the ones in count table and rename the gene names 

In [16]:
gene_length = read.table("../data/gene_length.tabular", h = F, row.names = 1)
gene_length = gene_length[rownames(counts),]
names(gene_length) = new_names
save(gene_length, file=paste(dp, "gene_length.RData", sep=''))

In [17]:
save(metadata, file=paste(dp, "metadata.RData", sep=''))

# Citations

In [18]:
citation("rentrez")


To cite rentrez in publications use:

  Winter, D. J. (2017) rentrez: an R package for the NCBI eUtils API
  The R Journal 9(2):520-526

A BibTeX entry for LaTeX users is

  @Article{,
    title = {{rentrez}: an R package for the NCBI eUtils API},
    author = {David J. Winter},
    journal = {The R Journal},
    year = {2017},
    volume = {9},
    issue = {2},
    pages = {520--526},
  }
